In [2]:
# load data
import glob
import os

from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

local_dir = "/Users/runshengsong/data/catVdog_less"
nb_epoch = 3
batch_size = 32

def __get_nb_files(directory):
    """Get number of files by searching local dir recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt


train_dir = os.path.join(local_dir, "train")
val_dir = os.path.join(local_dir, "val")

# set up parameters
nb_train_samples = __get_nb_files(train_dir)
nb_classes = len(glob.glob(train_dir + "/*"))
nb_val_samples = __get_nb_files(val_dir)
nb_epoch = int(nb_epoch)
batch_size = int(batch_size)

# data prep
train_datagen =  ImageDataGenerator(
    preprocessing_function = preprocess_input
  )

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
    )

# generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=batch_size)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    )

Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(train_generator,
                    nb_epoch=nb_epoch,
                    steps_per_epoch=nb_train_samples//batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples//batch_size,
                    class_weight='auto',
                    verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    nb_epoch=nb_epoch,
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples//batch_size,
                    class_weight='auto',
                    verbose=1)

/Users/runshengsong/anaconda3/envs/insight/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/runshengsong/anaconda3/envs/insight/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., verbose=1, validation_data=<keras_pre..., steps_per_epoch=18, epochs=3, validation_steps=18, class_weight="auto")`


Epoch 1/3
17/18 [===========================>..] - ETA: 7s - loss: 7.1643 

KeyboardInterrupt: 